In [1]:
# 1. Gerekli kütüphanelerin yüklenmesi
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier

In [2]:
# 2. Veri Yükleme
train_df = pd.read_csv("../train.csv")
test_df = pd.read_csv("../test.csv")

# Hedef ve özelliklerin ayrılması
target_col = "Fertilizer Name"
feature_cols = [col for col in train_df.columns if col not in ["id", target_col]]

In [3]:
# 3. Eğitim/Doğrulama Bölünmesi (Hold-out yöntemi, stratified)
train_data, val_data = train_test_split(train_df, test_size=0.2, 
                                       stratify=train_df[target_col], random_state=42)
print(f"Eğitim için {len(train_data)} örnek, Doğrulama için {len(val_data)} örnek ayrıldı.")

Eğitim için 600000 örnek, Doğrulama için 150000 örnek ayrıldı.


In [4]:
# 4. Özellik Mühendisliği: Kategorik kodlama (One-Hot Encoding)
cat_features = ["Soil Type", "Crop Type"]
num_features = [col for col in feature_cols if col not in cat_features]

# One-Hot encoder'ı eğitim verisi kategorilerine göre fit et
ohe = OneHotEncoder(sparse=False, handle_unknown="ignore")
ohe.fit(train_data[cat_features])

# Dönüştürme: eğitim, doğrulama ve test
X_train_cat = ohe.transform(train_data[cat_features])
X_val_cat   = ohe.transform(val_data[cat_features])
X_test_cat  = ohe.transform(test_df[cat_features])

# One-hot ile oluşan yeni sütun isimleri:
ohe_cols = ohe.get_feature_names_out(cat_features)

# One-hot sonuçlarını DataFrame'e çevirip orijinal indeksleri sıfırla
X_train_cat_df = pd.DataFrame(X_train_cat, columns=ohe_cols)
X_val_cat_df   = pd.DataFrame(X_val_cat, columns=ohe_cols)
X_test_cat_df  = pd.DataFrame(X_test_cat, columns=ohe_cols)

# Sayısal özellikleri olduğu gibi al
X_train_num = train_data[num_features].reset_index(drop=True)
X_val_num   = val_data[num_features].reset_index(drop=True)
X_test_num  = test_df[num_features].reset_index(drop=True)

# Sayısal ve one-hot özellikleri birleştir
X_train = pd.concat([X_train_num, X_train_cat_df], axis=1)
X_val   = pd.concat([X_val_num, X_val_cat_df], axis=1)
X_test  = pd.concat([X_test_num, X_test_cat_df], axis=1)

y_train = train_data[target_col].values
y_val   = val_data[target_col].values

TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'

In [ ]:
# 5. Model Eğitimi (XGBoost çok sınıflı)
# Hedef değişkeni sayısal koda çevir (XGBoost string ile de çalışabilir ama sayı vermek tercih edilir)
classes = sorted(train_df[target_col].unique())  # sınıfları alfabetik sırada alıyoruz
class_to_idx = {c: i for i, c in enumerate(classes)}
y_train_num = np.array([class_to_idx[c] for c in y_train])
y_val_num   = np.array([class_to_idx[c] for c in y_val])

# Model tanımlama
xgb_model = XGBClassifier(n_estimators=200, max_depth=6, learning_rate=0.1, 
                          objective="multi:softprob", num_class=len(classes),
                          eval_metric="mlogloss", use_label_encoder=False,
                          random_state=42)
# Modeli eğit (erken durdurma ile)
xgb_model.fit(X_train, y_train_num, 
              eval_set=[(X_val, y_val_num)], 
              early_stopping_rounds=20, verbose=False)

In [ ]:
# 6. Doğrulama üzerinde performans değerlendirme (MAP@3 hesaplama)
val_pred_prob = xgb_model.predict_proba(X_val)
def mapk_score(y_true, pred_probs, k=3):
    """Gerçek etiketler ve tahmin olasılık matrisine bakarak MAP@k skorunu hesaplar."""
    n = pred_probs.shape[0]
    # Her örnek için olasılıkları büyükten küçüğe sıralı ilk k sınıfın indeksi
    topk_preds = np.argsort(pred_probs, axis=1)[:, -k:][:, ::-1]
    # Ortalama average precision hesapla
    apk_sum = 0.0
    for true_label, pred_topk in zip(y_true, topk_preds):
        # true_label sayısal (class index) olmalı
        if true_label in pred_topk:
            rank = np.where(pred_topk == true_label)[0][0] + 1  # 1-indexed
            apk_sum += 1.0 / rank
    return apk_sum / n

val_map3 = mapk_score(y_val_num, val_pred_prob, k=3)
print(f"Doğrulama MAP@3 skoru: {val_map3:.5f}")

In [ ]:
# 7. Tüm eğitim verisiyle final modeli eğit ve test verisini tahmin et
final_model = XGBClassifier(n_estimators=xgb_model.best_iteration or 200,
                            max_depth=6, learning_rate=0.1,
                            objective="multi:softprob", num_class=len(classes),
                            use_label_encoder=False)
final_model.fit(pd.concat([X_train, X_val], axis=0), 
               np.concatenate([y_train_num, y_val_num], axis=0))

test_pred_prob = final_model.predict_proba(X_test)

# En yüksek 3 olasılıklı sınıfı seç
top3_idx = np.argsort(test_pred_prob, axis=1)[:, -3:][:, ::-1]  # her satır için en büyük 3 indeks
top3_labels = [" ".join([classes[idx] for idx in row]) for row in top3_idx]

In [ ]:
# 8. Submission dosyasını oluşturma
submission = pd.DataFrame({
    "id": test_df["id"],
    "Fertilizer Name": top3_labels
})
submission.to_csv("submission_v2.csv", index=False)
print(submission.head(5))